In [11]:
# Import modules
import six
import mule
import numpy as np
import xarray as xr
import os
from datetime import datetime

In [12]:
# Set some config
restart_filepath = '/g/data/p66/rml599/LandUse-tests/pre-industrial-essential.astart'
stash_master_filepath = '/g/data/access/projects/access/umdir/vn10.9/ctldata/STASHmaster/STASHmaster_A'
vegfrac_filepath = '/g/data/p66/rml599/LandUse-tests/vegfrac-dom-tree.nc' # step 2
stash_range = range(851, 860+1)
step2 = True
levels = 5

In [13]:
# Load the file
restart = mule.FieldsFile.from_file(restart_filepath)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/mule/stashmaster.py:259: UserWarning: 
Unable to load STASHmaster from version string, path does not exist
Path: $UMDIR/vn7.3/ctldata/STASHmaster/STASHmaster_A
Please check that the value of mule.stashmaster.STASHMASTER_PATH_PATTERN is correct for your site/configuration
  warnings.warn(msg)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/mule/validators.py:198: UserWarning: 
File: /g/data/p66/rml599/LandUse-tests/pre-industrial-essential.astart
Incorrect dataset_type (found 1, should be one of (3,))
  warnings.warn(msg)


In [14]:
def extract_field(restart, stash_code, num_levels):

    fields = list()
    
    for field in restart.fields:
        if field.lbuser4 == stash_code and field.lbuser5 in range(1, num_levels+1):
            fields.append(field)

    return fields

def to_numpy(field_list):
    data = None
    num_levels = len(field_list)

    for field in field_list:

        _data = field.get_data()

        if data is None:
            shape = (num_levels, *_data.shape)
            data = np.zeros(shape)

        ix = field.lbuser5 - 1
        data[ix, :, :] = _data

    return data

def to_dataarray(field_list):

    data = None
    num_levels = len(field_list)

    for field in field_list:

        _data = field.get_data()

        if data is None:
            shape = (num_levels, *_data.shape)
            data = np.zeros(shape)

        ix = field.lbuser5 - 1
        data[ix, :, :] = _data

    da = xr.DataArray(
        data,
        dims=('z','y','x')
    )
    
    return da
    
# Extract data from UM file.
def extract_data_array(restart, stash_code, num_levels=5, do_mean=True):

    data = None
    
    for field in restart.fields:
        if field.lbuser4 == stash_code and field.lbuser5 in range(1, num_levels+1):

            # First encounter, create the data object
            if data is None:
                shape = (num_levels, *field.get_data().shape)
                data = np.zeros(shape)

            ix = field.lbuser5 - 1
            data[ix, :, :] = field.get_data()

    # Create a data array
    da = xr.DataArray(
        data,
        dims=('z', 'y', 'x')
    )

    if do_mean is False:
        return da

    # Nan the fill value
    # fill_value = da.min()
    # Leaving this out makes ocean super negative and the data invisible (only use for plotting!!)
    # da = da.where(da != fill_value)
    
    da_mean = da.mean('z')

    return da_mean


In [15]:
# Get the vegetation fracion
vf = extract_field(restart, 216, levels)

# Get the missing value
# https://github.com/metomi/mule/blob/master/mule/docs/source/user_guide_basics.rst
fill_value = vf[0].bmdi

# # Convert to data array
vf = to_dataarray(vf)
vf_provided = xr.open_dataset(vegfrac_filepath)

vf_provided = vf_provided.squeeze(drop=True).rename(vegtype='z', longitude='x', latitude='y').fraction.isel(z=range(0, levels))

vfsum = vf.sum('z')

In [16]:
providers = dict()

# Extract/modify the stashcodes for 851-860
for stash_code in stash_range:
    
    print(stash_code)

    # Extract the data
    data = extract_field(restart, stash_code, num_levels=levels)

    # Convert it to data array
    data = to_dataarray(data)
    
    # Multiply the vegetation fraction
    data_w = data * vf

    # Sum across the dimension
    data_ws = data_w.sum('z')

    # Divide by the sum of the vegfrac, if nonzero
    data_wsd = xr.where(vfsum == 0.0, 0.0, data_ws / vfsum)

    # Re-establish the fill value
    data_wsd = data_wsd.where(data_wsd != fill_value).fillna(fill_value)

    # Create a data provider for this new field
    providers[stash_code] = mule.ArrayDataProvider(data_wsd.data[:]

851
852
853
854
855
856
857
858
859
860


In [17]:
# Modify the file
new_restart = restart.copy()

for field in restart.fields:

    stash_code = field.lbuser4
    level = field.lbuser5

    # If it is one of the fields we want to modify, do it
    if stash_code in providers.keys() and level <= levels:
        # new_restart.fields.append(operators[stash_code])
        print(f'Updating stash {stash_code} level {level}')
        field.set_data_provider(providers[stash_code])
    
    if stash_code in [216, 835] and level <= 5:
        print(f'Updating stash {stash_code} level {level}')
        provider = mule.ArrayDataProvider(vf_provided.data[level-1,:,:])
        field.set_data_provider(provider)
    
    new_restart.fields.append(field)

Updating stash 216 level 1
Updating stash 216 level 2
Updating stash 216 level 3
Updating stash 216 level 4
Updating stash 216 level 5
Updating stash 835 level 1
Updating stash 835 level 2
Updating stash 835 level 3
Updating stash 835 level 4
Updating stash 835 level 5
Updating stash 851 level 1
Updating stash 851 level 2
Updating stash 851 level 3
Updating stash 851 level 4
Updating stash 851 level 5
Updating stash 852 level 1
Updating stash 852 level 2
Updating stash 852 level 3
Updating stash 852 level 4
Updating stash 852 level 5
Updating stash 853 level 1
Updating stash 853 level 2
Updating stash 853 level 3
Updating stash 853 level 4
Updating stash 853 level 5
Updating stash 854 level 1
Updating stash 854 level 2
Updating stash 854 level 3
Updating stash 854 level 4
Updating stash 854 level 5
Updating stash 855 level 1
Updating stash 855 level 2
Updating stash 855 level 3
Updating stash 855 level 4
Updating stash 855 level 5
Updating stash 856 level 1
Updating stash 856 level 2
U

In [18]:
# Intercept the write function to disable validation
def to_file(self, output_file_or_path):
        """
        Write to an output file or path.

        Args:
            * output_file_or_path (string or file-like):
                An open file or filepath. If a path, it is opened and
                closed again afterwards.

        .. Note::
            As part of this the "validate" method will be called. For the
            base :class:`UMFile` class this does nothing, but sub-classes
            may override it to provide specific validation checks.

        """
        # Call validate - to ensure the file about to be written out doesn't
        # contain obvious errors.  This is done here before any new file is
        # created so that we don't create a blank file if the validation fails
        if isinstance(output_file_or_path, six.string_types):
            self.validate(filename=output_file_or_path, warn=True)
        else:
            self.validate(filename=output_file_or_path.name, warn=True)

        if isinstance(output_file_or_path, six.string_types):
            with open(output_file_or_path, 'wb') as output_file:
                self._write_to_file(output_file)
        else:
            self._write_to_file(output_file_or_path)

new_restart.to_file = to_file

In [19]:
# Write out the data.
stash_start, stash_end = stash_range[0], stash_range[-1]
timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')

output_filename = f'mean5_VFW_{stash_start}-{stash_end}_{timestamp}.astart'

# if step2:
#     output_filename = f'mean5_VFWDOM_{stash_start}-{stash_end}_{timestamp}.astart'    

output_dir = '/g/data/p66/bjs581'
output_filepath = f'{output_dir}/{output_filename}'

new_restart.to_file(new_restart, output_file_or_path=output_filepath)

print(output_filepath)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/mule/validators.py:198: UserWarning: 
File: /g/data/p66/bjs581/mean5_VFW_851-860_20240204_101944.astart
Incorrect dataset_type (found 1, should be one of (3,))
  warnings.warn(msg)


/g/data/p66/bjs581/mean5_VFW_851-860_20240204_101944.astart
